In [3]:
import mediapipe as mp
import numpy as np
import pandas as pd
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

from custom_pose_landmarks import CustomPoseLandmark

In [4]:
mp_pose = solutions.pose

custom = CustomPoseLandmark(mp_pose)

In [26]:
def prepare_dataframe():
    axes = ['X', 'Y', 'Z']
    names = custom.get_landmarks().values()
    column_names = [('_'.join([name, axis])).lower() for name in names for axis in axes]

    return pd.DataFrame(
        columns = ['timestamp'] + column_names,
        dtype = float
    )

In [2]:
mp_pose = solutions.pose

def landmark2array(landmark):
    return np.array(
        [
            landmark.x,
            landmark.y,
            landmark.z,
            landmark.visibility
        ]
    )

def array2landmark(array):
    return landmark_pb2.NormalizedLandmark(
        x=array[0],
        y=array[1],
        z=array[2],
        visibility=array[3]
    )

def get_custom_landmarks(indexes, landmarks):
    """
    
    """
    # Create customize landmarks list
    custom_landmarks = landmark_pb2.NormalizedLandmarkList()

    # Extend list by chosen landmarks
    custom_landmarks.landmark.extend(
        [landmarks.landmark[index] for index in indexes])

    # Calculate the coordinates of neck landmark
    left_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER])
    right_shoulder = landmark2array(
        landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER])
    neck = np.mean([left_shoulder, right_shoulder], axis=0)
    neck_landmark = array2landmark(neck)

    # Add neck landmark to custom list
    custom_landmarks.landmark.add().CopyFrom(neck_landmark)

    return custom_landmarks

In [13]:
# [name.lower() + '_' + axis for name in custom.get_landmarks().values() for axis in ('x', 'y', 'z')]

In [46]:
# source posibilities: path, int
# source = r'C:/Users/nemet/Desktop/XPC_2023/MVI_7011.mp4'
source = r'C:/Users/nemet/Desktop/XPC_2023/test8.mp4'
# source = 0

# background bool: Flase, True
background = True

# output posibilities: 'landmarks', 'mask', None
output = 'landmarks'

In [47]:
data = prepare_dataframe()

time = 0
cap = cv2.VideoCapture(source)

# Setup MediaPipe instance
with mp_pose.Pose(
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    enable_segmentation=True
) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        image_shape = image.shape


        # Camera condition -> if selfie or web camera is chosen then flip image
        if source == 0:
            image = cv2.flip(image, 1)
    
        # Recolor image for image processing
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Image MediaPipe processing -> detection
        results = pose.process(image)

        # Recolor back to BGR for visualization
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # create a fake background
        fake_background = np.zeros(shape=image_shape, dtype='uint8')
        # use a fake background if necessary
        if not background:
            image = fake_background

        # Image processing for different outputs
        if output == 'landmarks':
            default_landmarks = results.pose_landmarks

            if default_landmarks:
                time += 1

                # Get custom landmarks and create new connection
                custom_landmarks = get_custom_landmarks(chosen_indexes, default_landmarks)
                #custom_connections.add((0, len(custom_landmarks.landmark) - 1))

                # Prepare a single record storage
                record = np.array([time])

                for landmark in custom_landmarks.landmark:
                    # Extract pose landmarks coordinates and store as array
                    coordinates = landmark2array(landmark)[:3]
                    record = np.concatenate(
                        [
                            record,
                            coordinates
                        ]
                    )

                # Save pose landmarks coordinates in time as DataFrame
                data = pd.concat([data, pd.DataFrame([record], columns = column_names)], ignore_index=True)

                # Draw customize landmarks on image
                mp_drawing.draw_landmarks(
                    image,
                    landmark_list=custom_landmarks,
                    connections=connections,
                )


        elif output == 'mask':
            image = results.segmentation_mask
            if image is None:
                image = fake_background


        cv2.imshow(f'background: {background}, output: {output}', image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

        if not cap.isOpened():
            exit()
            
cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range